## Noise Augmentation

## Instrumental Broadening

## Shift Augmentation

In [26]:
import numpy as np
# given a set of vectors consisting of values, move all the values to the right or left by a certain amount

def shift_spectra(energy_interval, energy_shift, intensity):

	random_shift = np.random.uniform(-energy_shift, energy_shift, size = (intensity.shape[0], 1))
	shifted_energy = random_shift/energy_interval
	shifted_energy = shifted_energy.astype(int)
	intensity = intensity.copy()
	for idx, inten in enumerate(intensity):
		intensity[idx] = np.roll(inten, shifted_energy[idx])

	return intensity


intensity = np.sin(np.linspace(0, 1, 1000)) + np.random.normal(0, 0.1, 1000)
intensity = np.tile(intensity, (10, 1))
energy_interval = 0.1
energy_shift = 10
shifted_intensity = shift_spectra(energy_interval, energy_shift, intensity)

[[-76]
 [-56]
 [ 55]
 [-97]
 [-18]
 [ 41]
 [-41]
 [ 93]
 [-98]
 [ 43]]


## Total Augmentation pipeline